In [85]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

import random as rnd

In [73]:
team_names =     ['Utah Jazz', 'Sacramento Kings', 'Washington Wizards',
                  'Boston Celtics', 'Milwaukee Bucks', 'Oklahoma City Thunder',
                  'Chicago Bulls', 'Phoenix Suns', 'Philadelphia 76ers',
                  'New Orleans Pelicans', 'Charlotte Hornets', 'Los Angeles Lakers',
                  'Indiana Pacers', 'Toronto Raptors', 'Cleveland Cavaliers',
                  'Denver Nuggets', 'Minnesota Timberwolves', 'Brooklyn Nets',
                  'San Antonio Spurs', 'Dallas Mavericks', 'Houston Rockets',
                  'Detroit Pistons', 'Portland Trail Blazers', 'Atlanta Hawks',
                  'Golden State Warriors', 'Miami Heat', 'Los Angeles Clippers',
                  'New York Knicks', 'Memphis Grizzlies', 'Orlando Magic', 'League Average']

team_names_nocity = ['Jazz', 'Kings', 'Wizards', 'Celtics', 'Bucks', 'Thunder',
                  'Bulls', 'Suns', '76ers', 'Pelicans', 'Hornets', 'Lakers',
                  'Pacers', 'Raptors', 'Cavaliers', 'Nuggets', 'Timberwolves', 'Nets',
                  'Spurs', 'Mavericks', 'Rockets', 'Pistons', 'Trail Blazers', 'Hawks',
                  'Warriors', 'Heat', 'Clippers','Knicks', 'Grizzlies', 'Magic', 'League Average']

team_abbrs =     ['UTA', 'SAC', 'WAS', 'BOS', 'MIL', 'OKC', 'CHI', 'PHX', 'PHI',
                  'NOP', 'CHA', 'LAL', 'IND', 'TOR', 'CLE', 'DEN', 'MIN', 'BKN',
                  'SAS', 'DAL', 'HOU', 'DET', 'POR', 'ATL', 'GSW', 'MIA', 'LAC',
                  'NYK', 'MEM', 'ORL', 'NBA']

team_name_abbr_dict = {}
team_abbr_name_dict = {}
team_names_nocity_dict = {}
team_id_dict={}

for i in range(len(team_names)):
    team_name_abbr_dict[team_names[i]]=team_abbrs[i]
    
for i in range(len(team_names_nocity)):
    team_names_nocity_dict[team_names_nocity[i]]=team_abbrs[i]

for i in range(len(team_abbrs)):
    team_abbr_name_dict[team_abbrs[i]]=team_names[i]

In [70]:
url = 'https://stats.nba.com/stats/leaguegamelog'
headers= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
          'Referer': 'https://www.nba.com/'}
payload = {
    'Counter': '1000',
    'DateFrom': '',
    'DateTo': '',
    'Direction': 'DESC',
    'LeagueID': '00',
    'PlayerOrTeam': 'T',
    'Season': '2022-23',
    'SeasonType': 'Regular Season',
    'Sorter': 'DATE'}

jsonData = requests.get(url, headers=headers, params=payload).json()


rows = jsonData['resultSets'][0]['rowSet']
columns = jsonData['resultSets'][0]['headers']

df = pd.DataFrame(rows, columns=columns)
df.drop(['VIDEO_AVAILABLE'], axis=1, inplace=True)


df['FG2M'] = df.FGM - df.FG3M
df['FG2A'] = df.FGA - df.FG3A

df['FG2_PTS'] = df.FG2M * 2
df['FG3_PTS'] = df.FG3M * 3

df['FRAC_ATT_2PT'] = df.FG2A / df.FGA
df['FRAC_ATT_3PT'] = df.FG3A / df.FGA

df['FRAC_MK_2PT'] = df.FG2M / df.FGM
df['FRAC_MK_3PT'] = df.FG3M / df.FGM

df['FRAC_PTS_2PT'] = df.FG2_PTS / df.PTS
df['FRAC_PTS_3PT'] = df.FG3_PTS / df.PTS
df['FRAC_PTS_FT'] = df.FTM / df.PTS 

df['OPP_TEAM_ABBR'] = df['MATCHUP'].str.strip().str[-3:]
df['OPP_PTS'] = df['PTS'] - df['PLUS_MINUS']

df['MONTH'] = pd.DatetimeIndex(df['GAME_DATE']).month
df['YEAR'] = pd.DatetimeIndex(df['GAME_DATE']).year

def home_or_away(string):
    if string[4]=='@': return 'AWAY'
    elif string[4]=='v': return 'HOME'
df['HOME_AWAY'] = df['MATCHUP'].map(home_or_away)

conferences = {'GSW':'WEST', 'POR':'WEST', 'SAC':'WEST', 'UTA':'WEST', 'MIA':'EAST', 'DEN':'WEST', 'MIN':'WEST',
               'PHI':'EAST', 'NOP':'EAST', 'ORL':'EAST', 'MIL':'EAST', 'CHI':'EAST','DET':'EAST', 'TOR':'EAST',
               'PHX':'WEST', 'LAL':'WEST', 'ATL':'EAST', 'WAS':'EAST', 'MEM':'WEST', 'CLE':'EAST', 'LAC':'WEST',
               'BOS':'EAST', 'NYK':'EAST', 'IND':'EAST', 'CHA':'EAST', 'SAS':'WEST', 'HOU':'WEST', 'DAL':'WEST',
               'OKC':'WEST', 'BKN':'EAST'}

df['CONFERENCE'] = df['TEAM_ABBREVIATION'].apply(lambda x: conferences.get(x))
df['OPP_CONFERENCE'] = df['OPP_TEAM_ABBR'].apply(lambda x: conferences.get(x))

# ADD OPONENTS RECORD ON GAME DATE
# ADD TEAM RECORD ON GAME DATE

df['DATE_MATCHUP'] = df['GAME_DATE'].str[5:] + ' ' + df['MATCHUP'].str[4:]
#df['DATE_MATCHUP'] = df['GAME_DATE'] + ' ' + df['MATCHUP'].str[4:]

df.TEAM_NAME = df.TEAM_ABBREVIATION.apply(lambda x: team_abbr_name_dict.get(x))

df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,FRAC_PTS_3PT,FRAC_PTS_FT,OPP_TEAM_ABBR,OPP_PTS,MONTH,YEAR,HOME_AWAY,CONFERENCE,OPP_CONFERENCE,DATE_MATCHUP
0,22022,1610612755,PHI,Philadelphia 76ers,0022200770,2023-02-01,PHI vs. ORL,None,0,6,...,0.300000,0.300000,ORL,18,2,2023,HOME,EAST,EAST,02-01 vs. ORL
1,22022,1610612753,ORL,Orlando Magic,0022200770,2023-02-01,ORL @ PHI,None,0,7,...,0.500000,0.055556,PHI,20,2,2023,AWAY,EAST,EAST,02-01 @ PHI
2,22022,1610612763,MEM,Memphis Grizzlies,0022200771,2023-02-01,MEM vs. POR,None,0,7,...,0.315789,0.157895,POR,13,2,2023,HOME,WEST,WEST,02-01 vs. POR
3,22022,1610612757,POR,Portland Trail Blazers,0022200771,2023-02-01,POR @ MEM,None,0,6,...,0.230769,0.000000,MEM,19,2,2023,AWAY,WEST,WEST,02-01 @ MEM
4,22022,1610612743,DEN,Denver Nuggets,0022200768,2023-01-31,DEN vs. NOP,W,240,43,...,0.393443,0.163934,NOP,113,1,2023,HOME,WEST,EAST,01-31 vs. NOP


In [77]:
def gauss_game_sim(team1abbr, team2abbr):
    
    team1Score = int(round(rnd.gauss(df[df.TEAM_ABBREVIATION==team1abbr].PTS.mean(),df[df.TEAM_ABBREVIATION==team1abbr].PTS.std())\
                 +rnd.gauss(df[df.TEAM_ABBREVIATION==team2abbr].OPP_PTS.mean(),df[df.TEAM_ABBREVIATION==team2abbr].OPP_PTS.std()))/2)
    
    team2Score = int(round(rnd.gauss(df[df.TEAM_ABBREVIATION==team2abbr].PTS.mean(),df[df.TEAM_ABBREVIATION==team2abbr].PTS.std())\
                 +rnd.gauss(df[df.TEAM_ABBREVIATION==team1abbr].OPP_PTS.mean(),df[df.TEAM_ABBREVIATION==team1abbr].OPP_PTS.std()))/2)
    
    return team1Score, team2Score

In [80]:
def today_matchups():
    url = "https://www.espn.com/nba/scoreboard"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    date = soup.find('h3', class_="Card__Header__Title Card__Header__Title--no-theme").text

    games = soup.find_all("div", class_='Scoreboard__RowContainer flex flex-column flex-auto')
    games_data = []

    df = pd.DataFrame()

    for game in games:
        game_data = {}
        away_tm = game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[0]
        home_tm = game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[1]
        game_data['Date'] = date
        game_data["away_tm"] = away_tm.text
        #game_data["away_tm_abbr"] = team_names_nocity_dict[away_tm.text]
        game_data["home_tm"] = home_tm.text
        #game_data["home_tm_abbr"] = team_names_nocity_dict[home_tm.text]
        games_data.append(game_data)

    df = pd.DataFrame(games_data)
    
    print(soup.title.text)
    print(date)
    display(df)

today_matchups()

NBA Basketball Scores - NBA Scoreboard - ESPN
Wednesday, February 1, 2023


,Date,away_tm,home_tm
0,"Wednesday, February 1, 2023",Magic,76ers
1,"Wednesday, February 1, 2023",Trail Blazers,Grizzlies
2,"Wednesday, February 1, 2023",Nets,Celtics
3,"Wednesday, February 1, 2023",Thunder,Rockets
4,"Wednesday, February 1, 2023",Warriors,Timberwolves
5,"Wednesday, February 1, 2023",Kings,Spurs
6,"Wednesday, February 1, 2023",Raptors,Jazz
7,"Wednesday, February 1, 2023",Hawks,Suns
8,"Wednesday, February 1, 2023",Wizards,Pistons


In [ ]:
def gauss_game_sim(team1abbr, team2abbr):
    
    team1Score = int(round(rnd.gauss(df[df.TEAM_ABBREVIATION==team1abbr].PTS.mean(),df[df.TEAM_ABBREVIATION==team1abbr].PTS.std())\
                 +rnd.gauss(df[df.TEAM_ABBREVIATION==team2abbr].OPP_PTS.mean(),df[df.TEAM_ABBREVIATION==team2abbr].OPP_PTS.std()))/2)
    
    team2Score = int(round(rnd.gauss(df[df.TEAM_ABBREVIATION==team2abbr].PTS.mean(),df[df.TEAM_ABBREVIATION==team2abbr].PTS.std())\
                 +rnd.gauss(df[df.TEAM_ABBREVIATION==team1abbr].OPP_PTS.mean(),df[df.TEAM_ABBREVIATION==team1abbr].OPP_PTS.std()))/2)
    
    return team1Score, team2Score

In [88]:
def today_score_preds():
    url = "https://www.espn.com/nba/scoreboard"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    date = soup.find('h3', class_="Card__Header__Title Card__Header__Title--no-theme").text

    games = soup.find_all("div", class_='Scoreboard__RowContainer flex flex-column flex-auto')
    games_data = []

    df_out = pd.DataFrame()

    for game in games:
        game_data = {}
        
        away_tm = game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[0]
        home_tm = game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[1]
        
        away_sc_pred, home_sc_pred = gauss_game_sim(team_names_nocity_dict[away_tm.text], team_names_nocity_dict[home_tm.text])
        
        game_data['Date'] = date
        
        game_data["away_tm"] = away_tm.text
        game_data['away_sc_pred'] = away_sc_pred
        #game_data["away_tm_abbr"] = team_names_nocity_dict[away_tm.text]
        
        game_data["home_tm"] = home_tm.text
        game_data['home_sc_pred'] = home_sc_pred
        #game_data["home_tm_abbr"] = team_names_nocity_dict[home_tm.text]
        
        
        games_data.append(game_data)

    df_out = pd.DataFrame(games_data)
    
    print(soup.title.text)
    print(date)
    display(df_out)
    
today_score_preds()

NBA Basketball Scores - NBA Scoreboard - ESPN
Wednesday, February 1, 2023


,Date,away_tm,away_sc_pred,home_tm,home_sc_pred
0,"Wednesday, February 1, 2023",Magic,119,76ers,112
1,"Wednesday, February 1, 2023",Trail Blazers,97,Grizzlies,128
2,"Wednesday, February 1, 2023",Nets,110,Celtics,107
3,"Wednesday, February 1, 2023",Thunder,116,Rockets,109
4,"Wednesday, February 1, 2023",Warriors,124,Timberwolves,117
5,"Wednesday, February 1, 2023",Kings,119,Spurs,108
6,"Wednesday, February 1, 2023",Raptors,104,Jazz,120
7,"Wednesday, February 1, 2023",Hawks,107,Suns,118
8,"Wednesday, February 1, 2023",Wizards,112,Pistons,124
